In [ ]:
import tensorflow as tf
import numpy as np
import pickle
from tensorflow import keras
from matplotlib import pyplot
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

labels = ['Airplane', 'Automobile', 'Bird', 'Cat',
          'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']


In [ ]:
# check if GPU is detected:
len(tf.config.list_physical_devices('GPU')) > 0

In [ ]:
# Load dataset

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Preprocessing

x_train_16 = tf.keras.applications.vgg16.preprocess_input(x_train)
x_test_16 = tf.keras.applications.vgg16.preprocess_input(x_test)

x_train_19 = tf.keras.applications.vgg19.preprocess_input(x_train)
x_test_19 = tf.keras.applications.vgg19.preprocess_input(x_test)

x_train_resnet = tf.keras.applications.resnet.preprocess_input(x_train)
x_test_resnet = tf.keras.applications.resnet.preprocess_input(x_test)


# Split into train and validation

x_train_16, x_validation_16, y_train_16, y_validation_16 = train_test_split(
    x_train_16, y_train, train_size=0.8)

x_train_19, x_validation_19, y_train_19, y_validation_19 = train_test_split(
    x_train_19, y_train, train_size=0.8)

x_train_resnet, x_validation_resnet, y_train_resnet, y_validation_resnet = train_test_split(
    x_train_resnet, y_train, train_size=0.8)

In [ ]:
def drawConfMatrix(model, x_test, y_test, labels):
    predictions = model.predict(x_test, verbose=True)
    predictions = np.argmax(predictions, axis=1)

    confMat = ConfusionMatrixDisplay(confusion_matrix(y_test, predictions, normalize='true'), display_labels=labels)
    fig, ax = pyplot.subplots(figsize=(8,8))
    confMat.plot(ax = ax,  xticks_rotation = 'vertical')

def plot_accuracy(history, model_name):
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.title(model_name)
    plt.show()

def plot_loss(history, model_name):
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.title(model_name)
    plt.show()

def plot_history(history, model_name):
    plot_accuracy(history, model_name)
    plot_loss(history, model_name)

In [ ]:
def create_vgg_model(con_base, n_classes, optimizer, fine_tune=False):

    conv_base = con_base
    
    for layer in conv_base.layers:
        layer.trainable = fine_tune

    top_model = conv_base.output
    top_model = Flatten()(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dropout(0.5)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    model = Model(inputs=conv_base.input, outputs=output_layer)

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

def create_resnet_model(con_base, n_classes, optimizer, fine_tune=False):

    conv_base = con_base

    for layer in conv_base.layers:
        layer.trainable = fine_tune

    top_model = conv_base.output
    top_model = Flatten()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    model = Model(inputs=conv_base.input, outputs=output_layer)

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
input_shape = (32, 32, 3)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
n_classes = 10
n_epochs = 250
vgg16 = tf.keras.applications.VGG16(include_top=False,
                                    weights='imagenet',
                                    input_shape=input_shape)

vgg19 = tf.keras.applications.VGG19(include_top=False,
                                    weights='imagenet',
                                    input_shape=input_shape)

resnet = tf.keras.applications.ResNet50(include_top=False,
                                    weights='imagenet',
                                    input_shape=input_shape)

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

VGG 16 Pretrained - FULL

In [ ]:
# for i in range(5):
#     keras.backend.clear_session()

#     vgg_model = create_vgg_model(vgg16, n_classes, optimizer, fine_tune=False)
#     vgg_history = vgg_model.fit(x_train_16, y_train_16,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_16, y_validation_16),
#                             callbacks=[early_stop],
#                             verbose=1)
#     vgg_model.save('models/vgg16_/' + str(i))
#     with open('./train_history/vgg16_' + str(i), 'wb') as file_pi:
#         pickle.dump(vgg_history.history, file_pi)

VGG 19 Pretrained - FULL


In [ ]:
# for i in range(5):
#     keras.backend.clear_session()

#     vgg_model = create_vgg_model(vgg19, n_classes, optimizer, fine_tune=False)
#     vgg_history = vgg_model.fit(x_train_19, y_train_19,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_19, y_validation_19),
#                             callbacks=[early_stop],
#                             verbose=1)
#     vgg_model.save('models/vgg19_/' + str(i))
#     with open('./train_history/vgg19_' + str(i), 'wb') as file_pi:
#         pickle.dump(vgg_history.history, file_pi)

VGG 16 Pretrained & Fine Tuned - FULL


In [ ]:
# for i in range(5):
#     keras.backend.clear_session()
#     vgg16 = tf.keras.applications.VGG16(include_top=False,
#                                     weights='imagenet',
#                                     input_shape=input_shape)
#     vgg_model = create_vgg_model(vgg16, n_classes, optimizer, fine_tune=True)
#     vgg_history = vgg_model.fit(x_train_16, y_train_16,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_16, y_validation_16),
#                             callbacks=[early_stop],
#                             verbose=1)
#     vgg_model.save('models/vgg16ft_/' + str(i))
#     with open('./train_history/vgg16ft_' + str(i), 'wb') as file_pi:
#         pickle.dump(vgg_history.history, file_pi)

VGG 19 Pretrained & Fine Tuned - FULL


In [ ]:
# for i in range(5):
#     keras.backend.clear_session()
#     vgg19 = tf.keras.applications.VGG19(include_top=False,
#                                     weights='imagenet',
#                                     input_shape=input_shape)
#     vgg_model = create_vgg_model(vgg19, n_classes, optimizer, fine_tune=True)
#     vgg_history = vgg_model.fit(x_train_19, y_train_19,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_19, y_validation_19),
#                             callbacks=[early_stop],
#                             verbose=1)
#     vgg_model.save('models/vgg19ft_/' + str(i))
#     with open('./train_history/vgg19ft_' + str(i), 'wb') as file_pi:
#         pickle.dump(vgg_history.history, file_pi)

VGG 16 No pretraining - FULL

In [ ]:
# for i in range(5):
#     keras.backend.clear_session()
#     vgg16 = tf.keras.applications.VGG16(include_top=False,
#                                     weights=None,
#                                     input_shape=input_shape)
#     vgg_model = create_vgg_model(vgg16, n_classes, optimizer, fine_tune=True)
#     vgg_history = vgg_model.fit(x_train_16, y_train_16,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_16, y_validation_16),
#                             callbacks=[early_stop],
#                             verbose=1)
#     vgg_model.save('models/vgg16_clean_/' + str(i))
#     with open('./train_history/vgg16_clean_' + str(i), 'wb') as file_pi:
#         pickle.dump(vgg_history.history, file_pi)

VGG 19 No pretraining - FULL

In [ ]:
# for i in range(5):
#     keras.backend.clear_session()
#     vgg19 = tf.keras.applications.VGG19(include_top=False,
#                                     weights=None,
#                                     input_shape=input_shape)
#     vgg_model = create_vgg_model(vgg19, n_classes, optimizer, fine_tune=True)
#     vgg_history = vgg_model.fit(x_train_19, y_train_19,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_19, y_validation_19),
#                             callbacks=[early_stop],
#                             verbose=1)
#     vgg_model.save('models/vgg19_clean_/' + str(i))
#     with open('./train_history/vgg19_clean_' + str(i), 'wb') as file_pi:
#         pickle.dump(vgg_history.history, file_pi)

ResNet50 Pretrained - FULL

In [ ]:
# for i in range(5):
#     keras.backend.clear_session()

#     resnet_model = create_resnet_model(resnet, n_classes, optimizer, fine_tune=False)
#     resnet_history = resnet_model.fit(x_train_resnet, y_train_resnet,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_resnet, y_validation_resnet),
#                             callbacks=[early_stop],
#                             verbose=1)
#     resnet_model.save('models/resnet/' + str(i))
#     with open('./train_history/resnet_' + str(i), 'wb') as file_pi:
#         pickle.dump(resnet_history.history, file_pi)

ResNet50 No Pretraining - FULL

In [ ]:
# for i in range(5):
#     keras.backend.clear_session()
#     resnet = tf.keras.applications.ResNet50(include_top=False,
#                                     weights=None,
#                                     input_shape=input_shape)
#     resnet_model = create_resnet_model(resnet, n_classes, optimizer, fine_tune=True)
#     resnet_history = resnet_model.fit(x_train_resnet, y_train_resnet,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_resnet, y_validation_resnet),
#                             callbacks=[early_stop],
#                             verbose=1)
#     resnet_model.save('models/resnet_clean_/' + str(i))
#     with open('./train_history/resnet_clean_' + str(i), 'wb') as file_pi:
#         pickle.dump(resnet_history.history, file_pi)

ResNet50 Pretrained & unfrozen - FULL

In [ ]:
# for i in range(5):
#     keras.backend.clear_session()
#     resnet = tf.keras.applications.ResNet50(include_top=False,
#                                     weights='imagenet',
#                                     input_shape=input_shape)
#     resnet_model = create_resnet_model(resnet, n_classes, optimizer, fine_tune=True)
#     resnet_history = resnet_model.fit(x_train_resnet, y_train_resnet,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation_resnet, y_validation_resnet),
#                             callbacks=[early_stop],
#                             verbose=1)
#     resnet_model.save('models/resnet_ft_/' + str(i))
#     with open('./train_history/resnet_ft_' + str(i), 'wb') as file_pi:
#         pickle.dump(resnet_history.history, file_pi)

Model reader

In [ ]:
# folder_path = 'models/advd02/'
# accuracies = []

# for i in range(5):
#     model = keras.models.load_model(folder_path + str(i))
#     test_loss, test_accuracy = model.evaluate(x=x_test, y=y_test) 
#     accuracies.append(test_accuracy)

# avg = np.average(accuracies)
# std = np.std(accuracies)
# minimal = np.min(accuracies)
# maximal = np.max(accuracies)
# print(f"model: {folder_path}, avg: {avg}, std: {std}, min: {minimal}, max: {maximal}")
# print('==============')

Hisotry reader

In [ ]:
# filename = './train_history/adv_d02_0'
# #filename = './train_history/vgg16_0'

# with open(filename, "rb") as file_pi:
#     history = pickle.load(file_pi)

# plot_history(history, 'AdvancedConvModelDropout')

Regularization

In [ ]:
def create_advanced_model(dropout = 0.0):
    model = keras.Sequential()
    model.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Conv2D(filters=32,kernel_size=(4,4),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Conv2D(filters=32,kernel_size=(4,4),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

def create_advanced_model_with_reg(reg, dropout = 0.0):
    model = keras.Sequential()
    model.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu', kernel_regularizer=reg))
    model.add(MaxPool2D(pool_size=(2,2)))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Conv2D(filters=32,kernel_size=(4,4),activation='relu', kernel_regularizer=reg))
    model.add(MaxPool2D(pool_size=(2,2)))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Conv2D(filters=32,kernel_size=(4,4),activation='relu', kernel_regularizer=reg))
    model.add(MaxPool2D(pool_size=(2,2)))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(256,activation='relu', kernel_regularizer=reg))
    if dropout > 0.0:
        model.add(Dropout(dropout))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

early_stop = EarlyStopping(monitor='val_loss',
                           patience=20,
                           restore_best_weights=False,
                           mode='min')

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train / 255.0  # normalization
x_test = x_test / 255.0
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, train_size=0.8)

In [ ]:
# for i in range(5):
#     keras.backend.clear_session()
#     advanced_model =  create_advanced_model()
#     advanced_model_history = advanced_model.fit(x_train, y_train,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation, y_validation),
#                             callbacks=[early_stop],
#                             verbose=1)
#     advanced_model.save('models/adv/' + str(i))
#     with open('./train_history/adv' + str(i), 'wb') as file_pi:
#         pickle.dump(advanced_model_history.history, file_pi)

Dropout - FULL

In [ ]:
# for i in range(5):
#     keras.backend.clear_session()
#     advanced_model =  create_advanced_model(0.2)
#     advanced_model_history = advanced_model.fit(x_train, y_train,
#                             batch_size=32,
#                             epochs=500,
#                             validation_data=(x_validation, y_validation),
#                             callbacks=[early_stop],
#                             verbose=1)
#     advanced_model.save('models/advd02/' + str(i))
#     with open('./train_history/adv_d02_' + str(i), 'wb') as file_pi:
#         pickle.dump(advanced_model_history.history, file_pi)

# for i in range(5):
#     keras.backend.clear_session()
#     advanced_model =  create_advanced_model(0.1)
#     advanced_model_history = advanced_model.fit(x_train, y_train,
#                             batch_size=32,
#                             epochs=500,
#                             validation_data=(x_validation, y_validation),
#                             callbacks=[early_stop],
#                             verbose=1)
#     advanced_model.save('models/advd01/' + str(i))
#     with open('./train_history/adv_d01_' + str(i), 'wb') as file_pi:
#         pickle.dump(advanced_model_history.history, file_pi)

Regulizer L2 - FULL

In [ ]:
# from keras.regularizers import l2

# for i in range(5):
#     keras.backend.clear_session()
#     advanced_model =  create_advanced_model_with_reg(l2(0.01))
#     advanced_model_history = advanced_model.fit(x_train, y_train,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation, y_validation),
#                             callbacks=[early_stop],
#                             verbose=1)
#     advanced_model.save('models/adv_reg001/' + str(i))
#     with open('./train_history/adv_reg001_' + str(i), 'wb') as file_pi:
#         pickle.dump(advanced_model_history.history, file_pi)

# for i in range(5):
#     keras.backend.clear_session()
#     advanced_model =  create_advanced_model_with_reg(l2(0.001))
#     advanced_model_history = advanced_model.fit(x_train, y_train,
#                             batch_size=32,
#                             epochs=n_epochs,
#                             validation_data=(x_validation, y_validation),
#                             callbacks=[early_stop],
#                             verbose=1)
#     advanced_model.save('models/adv_reg001/' + str(i))
#     with open('./train_history/adv_reg001_' + str(i), 'wb') as file_pi:
#         pickle.dump(advanced_model_history.history, file_pi)

